In [1]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf

from rainforest_functions import vectorize_categories

## Here starts the program

In [2]:
mypath = '/Users/AnaSolaguren-Beascoa/Pictures/train-jpg/'

In [ ]:
pixel_df = pd.read_csv(mypath+'pixels_small_2.csv')
pixel_df = pixel_df.drop('Unnamed: 0',axis=1)

In [40]:
num_files = pixel_df.shape[0]

In [41]:
image_name_df = pd.read_csv('train_v2.csv')
image_name_df_0 = image_name_df.ix[0:num_files-1]

In [42]:
categories = image_name_df_0['tags'].str.split(' ', expand=True).stack().unique()

In [43]:
result = pd.merge(pixel_df, image_name_df_0, on='image_name')

In [44]:
X = result.drop(['tags','image_name'],axis=1)
y_all = result['tags']

In [45]:
vectorize_categories(categories, y_all)

In [46]:
y_vec_mine = pd.DataFrame(columns=['IF','NOTIF'], index=range(y_vec_all.shape[0]))
y_vec_mine['IF'] = y_vec_all

y_vec_mine.loc[y_vec_mine['IF'] == 1, 'NOTIF'] = 0
y_vec_mine.loc[y_vec_mine['IF'] == 0, 'NOTIF'] = 1

Lets symmetrise the data

In [17]:
#this is the data corresponding to fraud
data_df = pd.concat([X, y_vec_mine],axis=1)
data_IF_df = data_df[data_df.IF==1]
data_IF_df = data_IF_df.reset_index(drop=True)

#this is the data corresponding to non-fraud
data_NOTIF_df = data_df[data_df.IF==0]
data_NOTIF_df = data_NOTIF_df.reset_index(drop=True)
data_red_NOTIF_df = \
    data_NOTIF_df.ix[np.random.randint(1, max(data_NOTIF_df.index),max(data_IF_df.index)+1)]
data_red_NOTIF_df = data_red_NOTIF_df.reset_index(drop=True)

In [18]:
data_red_df = pd.concat([data_IF_df, data_red_NOTIF_df])
data_red_df = data_red_df.sample(frac=1).reset_index(drop=True)

X = data_red_df.drop(['IF', 'NOTIF'],axis=1)
y_vec_mine = pd.concat([data_red_df['IF'], data_red_df['NOTIF']],axis=1)

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_vec_mine, test_size=0.25)

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.fit_transform(X_test))

y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


Here we start to define our neural network

In [49]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],\
                        strides=[1, 2, 2, 1], padding='SAME')

In [50]:
lenx = X.shape[1]
#leny = len(categories)
leny = y_vec_mine.shape[1]


x = tf.placeholder(tf.float32, shape=[None, lenx])
y_ = tf.placeholder(tf.float32, shape=[None, leny])

x_image = tf.reshape(x, [-1, int(np.sqrt(lenx)), int(np.sqrt(lenx)), 1])
x_image = tf.cast(x_image, tf.float32)
#layer 1

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


#layer 2
W_conv2 = weight_variable([20, 20, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#layer 3
W_fc1 = weight_variable([int(np.sqrt(lenx)/4)*int(np.sqrt(lenx)/4)*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, int(np.sqrt(lenx)/4)*int(np.sqrt(lenx)/4)*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#layer 4
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#layer 5
W_fc2 = weight_variable([1024, leny])
b_fc2 = bias_variable([leny])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#output
y = y_conv


In [51]:
X_train.shape[0]

30359

In [78]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
#train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

steps = 1000
batchsize = int(X_train.shape[0]/steps)
i = 0
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for k in range(steps):
    batch_xs = X_train.iloc[i:i+batchsize,:].as_matrix().astype(np.float32)
    batch_ys = y_train.iloc[i:i+batchsize].as_matrix().astype(np.float32)
    if k % 100 == 0:
        train_accuracy = sess.run(accuracy,feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0})
        print('step %d, training accuracy %g' % (k, train_accuracy))
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
            
    i += batchsize
    

step 0, training accuracy 0.3
step 100, training accuracy 0.366667
step 200, training accuracy 0.633333
step 300, training accuracy 0.7
step 400, training accuracy 0.666667
step 500, training accuracy 0.7
step 600, training accuracy 0.633333
step 700, training accuracy 0.7
step 800, training accuracy 0.7
step 900, training accuracy 0.633333


In [76]:
print(sess.run(accuracy, feed_dict={x: X_test.iloc[:5,:], y_: batch_ys, keep_prob: 1.0}))

<tf.Tensor 'Mean_14:0' shape=() dtype=float32>

In [85]:
answer = sess.run(y, feed_dict={x: X_train.iloc[:6,:], keep_prob: 1})
answer

array([[ -41.65432358,  -67.79103851],
       [ -51.12691879,   21.2522049 ],
       [ -71.39070129,   55.69518661],
       [ -99.86793518,   89.54359436],
       [-138.02716064, -109.19327545],
       [ -60.01911545,   21.65198135]], dtype=float32)

In [ ]:
answer = sess.run(y, feed_dict={x: X_test.as_matrix()})
answer


In [ ]:
for i in range(len(answer)):
    if answer[i][0]>answer[i][1]:
        answer[i][0]=1
        answer[i][1]=0
    else:
        answer[i][1]=1
        answer[i][0]=0
        

In [ ]:
answer

In [ ]:
from sklearn.metrics import accuracy_score, fbeta_score


accuracy_score(pd.DataFrame(answer)[0], y_test['IF'])

In [ ]:
print(fbeta_score(pd.DataFrame(answer)[0], y_test['IF'],beta=2, average='weighted'))

In [ ]:
answer_train = sess.run(y, feed_dict={x: X_train.as_matrix()})
for i in range(len(answer_train)):
    if answer_train[i][0]>answer_train[i][1]:
        answer_train[i][0]=1
        answer_train[i][1]=0
    else:
        answer_train[i][1]=1
        answer_train[i][0]=0
        

print(fbeta_score(pd.DataFrame(answer_train)[0], y_train['IF'],beta=2, average='weighted'))

In [ ]:
pd.DataFrame(answer_train)[0].shape

In [ ]:
y_train['IF'].shape